In [1]:
# pip install python-dotenv
# pip install psycopg2-binary

In [21]:
import os
import requests
from dotenv import load_dotenv
import psycopg2
from bs4 import BeautifulSoup
import re
import time

#env 파일 환경변수 로드
load_dotenv() 

# KaKao API 설정
KAKAO_API_KEY = os.getenv("KAKAO_REST_API_KEY")
headers = { "Authorization": f"KakaoAK {KAKAO_API_KEY}" }
url = "https://dapi.kakao.com/v2/local/search/keyword.json"

In [22]:
# PostgreSQL 연결 설정
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    cur = conn.cursor()
    print("DB connect 성공")
except Exception as e:
    print("실패",e)

DB connect 성공


In [4]:
def save_cafe_db(cafe):
    
    cur.execute("""
        INSERT INTO cafe (kakao_id, name, address, road_address, phone,
        category, longitude, latitude, place_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (kakao_id) DO NOTHING;
                """,(
                    cafe['id'], cafe['place_name'], cafe['address_name'],cafe['road_address_name'],
                    cafe['phone'], cafe['category_group_name'], float(cafe['x']), float(cafe['y']),
                    cafe['place_url']
                ))
    conn.commit()
    
def fetch_all_cafe(query="강남 카페"):
    all_cafes = []
    for page in range(1,4):
        params={
            "query": query,
            "size":15,
            "page":page
            }
        res = requests.get(url,headers=headers,params=params)
        data = res.json()
    
        cafes = data['documents']
        if not cafes:
            break
        all_cafes.extend(cafes)
    return all_cafes
    
    
# 실행

cafe_region = ["강남 카페","홍대 카페","신촌 카페","이대 카페","이태원 카페","성수 카페","연남동 카페",\
    "한남동 카페","압구정 카페","청담 카페","종로 카페","인사동 카페","건대입구 카페"]

for cafe_r in cafe_region:
    cafes = fetch_all_cafe(cafe_r)
    for cafe in cafes:
        save_cafe_db(cafe)
    
    print(f"저장 완료{len(cafes)}개 수집됨")
    

저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨


In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



def crawl_place_info(place_url):
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    try:
        driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(place_url)
        
        print("브라우저 실행 성공")
        wait = WebDriverWait(driver,5)
    
        blog_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'a.link_blog')))
        text = blog_element.text
        match = re.search(r'블로그 리뷰\s*(\d+)',text)
        
        if match:
            print("크롤링 성공",match.group(1))
            return int(match.group(1))
        else:
            print("리뷰 없음")
            return 0
        
    except Exception as e:
        print("크롤링 실패",e)
        return None
    
    finally:
        driver.quit()

cur.execute("SELECT id, place_url FROM cafe")
rows = cur.fetchall()

for cafe_id, place_url in rows:
    count = crawl_place_info(place_url)
    if count is None:
        count = 0
    print(f"[{cafe_id}] 블로그 리뷰 수: {count}")
    
    time.sleep(1)    

브라우저 실행 성공
리뷰 없음
[1] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[2] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[3] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[4] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[5] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[6] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[7] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[8] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[9] 블로그 리뷰 수: 0
브라우저 실행 성공
리뷰 없음
[10] 블로그 리뷰 수: 0


UnboundLocalError: cannot access local variable 'driver' where it is not associated with a value